In [ ]:
CREATE OR REPLACE WAREHOUSE Fraud_wh WITH
WAREHOUSE_SIZE='X-SMALL'
AUTO_SUSPEND = 120
AUTO_RESUME = TRUE
INITIALLY_SUSPENDED=TRUE;

In [ ]:
create database if not exists Fraud_db;


CREATE COMPUTE POOL Fraud_cp
MIN_NODES = 1 MAX_NODES = 1 INSTANCE_FAMILY = CPU_X64_L 
AUTO_RESUME = TRUE INITIALLY_SUSPENDED = TRUE AUTO_SUSPEND_SECS = 300; 

In [ ]:
cust_spdf=Generate_Customer_Features.to_df()

cust_spdf.write.mode('overwrite').save_as_table('customer_features')
cust_sdf=session.sql("select * from customer_features")


trans_df=Generate_Transaction_Features.to_df()

trans_df.write.mode('overwrite').save_as_table('transaction_features')
trans_sdf=session.sql("select * from transaction_features")


managed_fv = FeatureView(
    name="Customer_Features",
    entities = [customer_entity],
    feature_df=cust_sdf,
    overwrite=True
)
registered_fv = FS1.register_feature_view(
    feature_view=managed_fv,
    version="V1"
)


managed_fv = FeatureView(
    name="Trans_Features",
    entities = [transaction_entity],
    feature_df=trans_sdf,
    overwrite=True
    
)

registered_fv = FS1.register_feature_view(
    feature_view=managed_fv,
    version="V1"
)

NameError: name 'Generate_Customer_Features' is not defined

In [ ]:

# Access feature views

fs = FeatureStore(
    session=session,
    database="CC_FINS_DB",
    name="ANALYTICS",
    default_warehouse="CC_FINS_WH",
    creation_mode=CreationMode.FAIL_IF_NOT_EXIST
)

customer_fv : FeatureView = fs.get_feature_view(
    name='Customer_Features',
    version='V1'
)
print(customer_fv)

trans_fv : FeatureView = fs.get_feature_view(
    name='Trans_Features',
    version='V1'
)
print(trans_fv)


In [ ]:
# Get transactions dataset and get features from the feature store
def create_dataset(spine_df, name):
    train_dataset = fs.generate_dataset(
    name=name,
    spine_df=spine_df,
    features=[customer_fv, trans_fv]
    )
    df = train_dataset.read.to_snowpark_dataframe()
    return df
# Split into train/validation/test


datasets = TRANSACTIONS_DATA_df.random_split([.8,.2])

# Build training tables
train_df = create_dataset(datasets[0], "train")
val_df = create_dataset(datasets[1], "validation")

